In [25]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import CocoDataset2
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import torch.optim as optim

# Define a convolution neural network
class CNN(nn.Module):
    def __init__(self, dropout_rate =0.5, weight_decay=1e-4):
        super(CNN, self).__init__()
        #include the batch normalization and max pooling in accounting for overfitting
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(80736, 10)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.weight_decay = weight_decay

    def forward(self, input):
        #compute value of the loss function
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))     
        output = self.flatten(output)
        print("Flattened output shape:", output.shape)

        output = self.dropout(output)
        output = self.fc1(output)

        return output


In [3]:
def collate_fn(batch):
    images, annotations = zip(*batch)

    # Process annotations to extract labels (assuming "labels" is the key)
    labels = [a["labels"] for a in annotations]

    # Stack images and labels into tensors
    images = torch.stack(images)
    labels = torch.cat(labels)

    return images, labels

In [14]:
transformations = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((128, 128)),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [29]:
# Load MS COCO dataset (change the data path accordingly)
coco_dataset = CocoDataset2.CocoDataset(
    root_path= "C:/Users/trand/Turion Space Challenge/coco2017/train2017",
    annotation_file= "C:/Users/trand/Turion Space Challenge/coco2017/annotations/instances_train2017.json", transforms=transformations
)

# Create a DataLoader
data_loader = DataLoader(coco_dataset, batch_size=32, collate_fn=collate_fn)



loading annotations into memory...
Done (t=10.93s)
creating index...
index created!


In [16]:
print(len(coco_dataset))

118287


In [27]:
# Initialize the model, loss function, and optimizer
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
optimizer2 = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)



In [18]:
def saveModel():
    path = "./ModelCOCO.pth"
    torch.save(model.state_dict(), path)

In [19]:
import matplotlib.pyplot as plt
import numpy as np

def show_loss(loss_list):
    epochs = np.arange(1, len(loss_list)+1)
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss_list, label='Training Loss', marker='o')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [21]:
def plot_uncertainty(pred_mean, pred_variance, epoch):
    # Plot the predictive mean

    # Generate x-axis values (epochs)
    classes = np.arange(1, 11)
    new_pred_mean = np.array(pred_mean)
    new_pred_variance = np.array(pred_variance)
    # Plot predictive mean
    plt.plot(classes, new_pred_mean, marker='o', linestyle='-', color='b', label='Predictive Mean')

    # Plot uncertainty as a filled area
    plt.fill_between(classes, new_pred_mean - np.sqrt(new_pred_variance), new_pred_mean + np.sqrt(new_pred_variance),
                 color='blue', alpha=0.2, label='Uncertainty Estimation')

    # Add labels and title
    plt.xlabel('Class')
    plt.ylabel('Predictive Mean and Uncertainty')
    plt.title('Predictive Mean and Uncertainty Estimation Over Epochs ' + str(epoch+1))
    plt.legend()

    # Show the plot
    plt.show()

In [28]:

best_accuracy = 0.0
train_loss = []

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0

    for i, (images, labels) in enumerate(data_loader):
        
        # get the inputs
        # images = Variable(images.to(device))
        # labels = Variable(labels.to(device))

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images)
        # compute the loss based on model output and real labels
        loss = loss_fn(outputs, labels)
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        if i % 1000 == 999:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 1000))
            # zero the loss
            running_loss = 0.0
    avg_train_loss =  running_loss / len(train_loader)
    print('average training loss: ' + str(avg_train_loss))
    train_loss.append(avg_train_loss)
       
    model.eval()
    accuracy = 0.0
    total = 0.0
    probs = []
    max_len = 0
    #compare with the test set 
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()

            #softmax due to our image classification task 
            probs.append(outputs.softmax(dim=1).numpy())
            max_len = max(max_len, outputs.shape[1])
    #give the percentage accuracy 
    accuracy = (100 * accuracy / total)
    print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
    
    l = 8
    N = 118287
    for i in range(len(probs)):
        current_len = probs[i].shape[0]
        if current_len < max_len:
            # Pad with zeros
            padding = max_len - current_len
            probs[i] = np.concatenate((probs[i], np.zeros((padding,))))
        elif current_len > max_len:
            # Trim
            probs[i] = probs[i][:max_len]
    #will produce a list with 10 lists each present the predictive mean for each class
    predictive_mean = np.mean(probs, axis=0)
    #similar to the predictive mean
    predictive_variance = np.var(probs, axis=0)
    tau = l**2 * (1 - model.dropout.p) / (2 * N * model.weight_decay)
    predictive_variance += tau**-1

    print("Predictive mean for epoch", epoch+1, len(predictive_mean[0]))
    print("Uncertainty estimation for epoch", epoch+1, len(predictive_variance[0]))
    #plot the uncertainty over predicitve mean on each epochs (iterations **)
    plot_uncertainty(predictive_mean[0], predictive_variance[0], epoch)

    if accuracy > best_accuracy:
        saveModel()
        best_accuracy = accuracy



Flattened output shape: torch.Size([16, 80736])


ValueError: Expected input batch_size (16) to match target batch_size (72).